Importing all essential python libraries

In [1]:
import numpy as np
import pandas as pd

#for getting world map
import folium

Scraping Latitude and Longitude from quicksgs.com

In [2]:
# read_html() method returns all the tables in the webpage (Internally beautifulSoup is executed)

# Retreiving Latitude and Longitude coordinates
info = pd.read_html("http://www.quickgs.com/latitudinal-and-longitudinal-extents-of-india-indian-states-and-cities/") 

#convering the table data into DataFrame
coordinates = pd.DataFrame(info[0])

coordinates.head()

,State,Latitude,Longitude
0,Andhra Pradesh,15.91°N,79.74°E
1,Arunachal Pradesh,28.21°N,94.72°E
2,Assam,26.20°N,92.93°E
3,Bihar,25.09°N,85.31°E
4,Chhattisgarh,21.27°N,81.86°E


Scraping live corona stats in INDIAN states from Wikipedia

In [3]:
coronastats = pd.read_html('https://en.wikipedia.org/wiki/COVID-19_pandemic_in_India#covid19-container',match='State/Union Territory')

#Convert to DataFrame
covid19 = pd.DataFrame(coronastats[0])

covid19.head()

[show all]viewtalkeditCOVID-19 pandemic in India by state and union territory  ...                   
                                                          State/Union Territory  ... Unnamed: 8_level_1
                                                                        35 / 36  ... Unnamed: 8_level_2
0                        Andaman and Nicobar Islands                             ...                NaN
1                                     Andhra Pradesh                             ...                NaN
2                                  Arunachal Pradesh                             ...                NaN
3                                              Assam                             ...                NaN
4                                              Bihar                             ...                NaN

[5 rows x 9 columns]

Data Cleaning and preparation

In [4]:
def data_pre(cord):
    return cord[0:5]
coordinates['Latitude']  = coordinates['Latitude'].apply(data_pre).astype('float')
coordinates['Longitude'] = coordinates['Longitude'].apply(data_pre).astype('float')
coordinates.head()

# cleaning Covid19 Dataframe

# Removing unnecessary rows at the tail and Unnecessary columns
covid19 = covid19.iloc[:-3,:-4]

# Renaming Attribute Names for simplicity
covid19.columns = ['State','Total cases','Deaths','Recoveries','Active cases']

covid19.head()

,State,Total cases,Deaths,Recoveries,Active cases
0,Andaman and Nicobar Islands,3557,52,3278,227
1,Andhra Pradesh,575079,4972,476903,93204
2,Arunachal Pradesh,6298,11,4531,1756
3,Assam,"144,166[b]",482,115054,28630
4,Bihar,160366,831,145560,13975


In [5]:
#Merging the tables covid19 and coordinates 
final_data = pd.merge(coordinates, covid19, how ='inner', on ='State')
final_data.head()

,State,Latitude,Longitude,Total cases,Deaths,Recoveries,Active cases
0,Andhra Pradesh,15.91,79.74,575079,4972,476903,93204
1,Arunachal Pradesh,28.21,94.72,6298,11,4531,1756
2,Assam,26.20,92.93,"144,166[b]",482,115054,28630
3,Bihar,25.09,85.31,160366,831,145560,13975
4,Chhattisgarh,21.27,81.86,67327,573,33109,33645


Here we go with the final step to Visualize this data!

we use folium (python library) to depict all the data above on the INDIA map.

In [7]:
india = folium.Map(location = [20.5937,78.9629],zoom_start=4.5)
#adding to map
for state,lat,long,total_cases,Death,Recov,Active in zip(list(final_data['State']),list(final_data['Latitude']),list(final_data['Longitude']),list(final_data['Total cases']),list(final_data['Deaths']),list(final_data['Recoveries']),list(final_data['Active cases'])):
    #for creating circle marker
    folium.CircleMarker(location = [lat,long],
                       radius = 5,
                       color='red',
                       fill = True,
                       fill_color="red").add_to(india)
    #for creating marker
    folium.Marker(location = [lat,long],
                  # adding information that need to be displayed on popup
                  popup=folium.Popup(('<strong><b>State  : '+state+'</strong> <br>' +
                    '<strong><b>Total Cases : '+total_cases+'</striong><br>' +
                    '<strong><font color= red>Deaths : </font>'+Death+'</striong><br>' +
                    '<strong><font color=green>Recoveries : </font>'+Recov+'</striong><br>' +
                    '<strong><b>Active Cases : '+Active+'</striong>' ),max_width=200)).add_to(india)
    #to show the map

In [8]:
india